In [ ]:
!pip install ultralytics==8.0.196 --quiet
!pip install matplotlib Pillow --quiet

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from google.colab import drive
import zipfile
import os


In [ ]:
drive.mount('/content/drive')

In [ ]:
# Replace 'your_zip_file.zip' with the name of your zip file
zip_file_path = '/content/drive/MyDrive/Models/Segmentation/Model-m/model-m.zip'

# Replace 'extraction_path' with the path where you want to extract the contents
extraction_path = '/content/'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [ ]:
# Load the YOLOv8 model
model = YOLO('/content/best.pt')

# Open the video file
video_path = '/content/drive/MyDrive/dataset_1/Video_4Trim.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create a VideoWriter object to save the processed video
output_path = '/content/output_video2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Iterate through each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to PIL image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Perform inference on the frame
    results = model(pil_image)

    # Extract bounding boxes, classes, names, and confidences
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()

    # Plot bounding boxes and labels for the "open" and "Closed" classes
    for box, cls, conf in zip(boxes, classes, confidences):
        x1, y1, x2, y2 = box

        if names[int(cls)] == "open":
            # Draw bounding box for "open" class
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            # Add label for "open" class
            cv2.putText(frame, f'{names[int(cls)]} {conf:.2f}', (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            print("Bounding Box Coordinates for the box:")
            print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")

        elif names[int(cls)] == "Closed":
            print('Box Closed')

    # Display the frame
    cv2_imshow(frame)

    # Write the frame to the output video
    out.write(frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()